In [1]:
import os
import glob
from datetime import datetime
from netCDF4 import Dataset
import netCDF4 as nc
from datetime import datetime
import pandas as pd
import numpy as np
import xarray as xr
import sys
sys.path.append('C:/Users/Administrator/anaconda3/envs/gdal/Lib/site-packages/')
sys.path.append('C:/Users/Administrator/anaconda3/envs/gdal/Lib/site-packages/lmoments3')
import distr
from scipy import stats
import matplotlib.pyplot as plt
import multiprocessing as mp
from tqdm import tqdm
import basefunc

In [2]:
'''read_data'''

def read_data(scen,time_low,time_up):
    '''
    scen: scenario ('historical', 'ssp126' or 'ssp245','ssp370,'ssp585')
    time_low: start year (1985 for historical, 2030 for mid-21 century, 2070 for late-21 century)
    time_up: end year (2014 for historical, 2059 for mid-21 century, 2099 for late-21 century)
    '''    
    
    
    model = ['ACCESS-CM2','ACCESS-ESM1-5']
    #model.remove('GFDL-CM3')
    for i in range(len(model)):
        path=os.path.join(f'D:/QQ/PhD/cmip6_split/{model[i]}_Rx1d/',scen)
        
        all_files=os.listdir(path)
        all_files.sort()
        
        # filt files for specific years
        files=[]
        for f in range(len(all_files)):
            if int(all_files[f][-8:-4])>=time_low and int(all_files[f][-8:-4])<=time_up:
                files.append(all_files[f])      
                
        # concatenate files for 30 years        
        for j in range(len(files)):
            data=basefunc.getRaster(os.path.join(path, files[j]))
            data=data.reshape(1,data.shape[0],data.shape[1])
            if j==0:
                climate_model_data=data
            else:
                climate_model_data=np.concatenate((climate_model_data,data),axis=0)  
                
        # concatenate files for 20 models        
        climate_model_data=climate_model_data.reshape(1,climate_model_data.shape[0],climate_model_data.shape[1],climate_model_data.shape[2])       
        if i==0:              
            climate_data=climate_model_data
        else:
            climate_data=np.concatenate((climate_data,climate_model_data),axis=0)    
        del climate_model_data
        del data
    return climate_data,model


def fit_gev_at_point(point_data):

    rt= np.array([1.1,2,5,10,20,30,50,100,200])
    if point_data.max()>=1e+10:
        return np.array([-1]*len(rt))
    else:
        ranseq=np.arange(10e-9,(3*10e-8+10e-9),10e-9)
        gevfit = distr.gev.lmom_fit(point_data+ranseq)
        fitted_gev = distr.gev(**gevfit) 
        D,p=stats.kstest(point_data,fitted_gev.cdf)
        if p>0.05:
            gevST = fitted_gev.ppf(1.0-1./rt)
            return gevST
        else:
            return np.array([-1]*len(rt))



def fit_gev_pre_point(point_data):
    
    data_fit=np.array(point_data[:30])
    data_convert=point_data[-9:]
    rt= np.array( [1.1,2,5,10,20,30,50,100,200] )
    if point_data.max()>=1e+10:
        gevST=np.array([-1]*len(rt))
        RT=np.array([-1]*len(rt))
        return np.concatenate((gevST,RT))
    else:
        ranseq=np.arange(10e-9,(3*10e-8+10e-9),10e-9)
        gevfit = distr.gev.lmom_fit(data_fit+ranseq)
        fitted_gev = distr.gev(**gevfit) 
        D,p=stats.kstest(point_data,fitted_gev.cdf)

        if p>0.05:
            gevST = fitted_gev.ppf(1.0-1./rt)
            RT =1/( 1-fitted_gev.cdf(data_convert))
            RT[np.isinf(RT)]=1
            return np.concatenate((gevST,RT))
        else:
            gevST=np.array([-1]*len(rt))
            RT=np.array([-1]*len(rt))
            return np.concatenate((gevST,RT))


In [4]:
scen = 'historical'

climate_data,model=read_data(scen,1985, 2015)
climate_data=np.sort(climate_data,axis=1)
mme_median=np.median(climate_data,axis=0)
print(mme_median.shape)
rs = np.empty((9,mme_median.shape[1], mme_median.shape[2]))
for i in tqdm(range(mme_median.shape[1])):
    for j in range(mme_median.shape[2]):
        point_data = mme_median[:, i, j]
        rs[:,i,j] = fit_gev_at_point(point_data)
ref_tif = './data/ref.tif'
output_path = f'./results_for_gev/{scen}_mme.tif'
basefunc.array2Raster(rs,ref_tif,output_path)

(30, 600, 1440)


100%|██████████| 600/600 [02:38<00:00,  3.79it/s]


In [5]:
SCENARIO_YEARS = {
    'ssp126': [(2030, 2060), (2070, 2100)],
    'ssp245': [(2030, 2060), (2070, 2100)],
    'ssp370': [(2030, 2060), (2070, 2100)],
    'ssp585': [(2030, 2060), (2070, 2100)],
}
ref_tif = './data/ref.tif'
historical_mme=basefunc.getRaster(r'./results_for_gev/historical_mme.tif')

for scenario, ranges in SCENARIO_YEARS.items():
    for time_low,time_up in ranges:
        print(scenario,time_low,time_up)
        climate_data,model=read_data(scenario,time_low,time_up)
        climate_data=np.sort(climate_data,axis=1)
        mme_median=np.median(climate_data,axis=0)
        del climate_data

        inputs=np.concatenate((mme_median,historical_mme),axis=0)
        rs = np.empty((18,inputs.shape[1], inputs.shape[2]))

        for i in tqdm(range(inputs.shape[1])):
            for j in range(inputs.shape[2]):
                point_data = inputs[:, i, j]
                rs[:,i,j] = fit_gev_pre_point(point_data)

        future_pre=rs[:9,:,:]
        future_rt=rs[-9:,:,:]
        
        outname=scenario+'pre'+str(time_low)+str(time_up)+'_mme.tif'
        output_path='./results_for_gev/'+outname
        basefunc.array2Raster(future_pre,ref_tif,output_path)

        outname='newRT_under_'+scenario+'_'+str(time_low)+str(time_up)+'_mme.tif'
        output_path='./results_for_gev/'+outname
        basefunc.array2Raster(future_rt,ref_tif,output_path)


ssp126 2030 2060


  3%|▎         | 16/600 [00:00<00:03, 153.17it/s]C:\Users\Administrator\AppData\Local\Temp\ipykernel_9888\2337039357.py:80: RuntimeWarning: divide by zero encountered in divide
  RT =1/( 1-fitted_gev.cdf(data_convert))
100%|██████████| 600/600 [03:43<00:00,  2.69it/s]


ssp126 2070 2100


100%|██████████| 600/600 [05:35<00:00,  1.79it/s]


ssp245 2030 2060


...

100%|██████████| 600/600 [05:51<00:00,  1.70it/s]


ssp585 2030 2060


100%|██████████| 600/600 [03:37<00:00,  2.76it/s]


ssp585 2070 2100


100%|██████████| 600/600 [03:21<00:00,  2.98it/s]
